In [1]:
%matplotlib inline
import numpy
import math
import pyparticleest.models.nlg as nlg
import pyparticleest.simulator as simulator
import matplotlib.pyplot as plt
import scipy
import pyparticleest.utils.kalman as kalman
from scipy.stats import bernoulli

Falling back to pure python implementaton, expect horrible performance


In [ ]:
class StdNonLin(nlg.NonlinearGaussianInitialGaussian):
    # x_{k+1} = x_k  + v_k = f(x_k) + v_k,
    # y_k = 0 + e_k = g(x_k) + e_k,
    # x(0) ~ N(0,P0), v_k ~ N(0,Q), e_k ~ Bernoulli(\exp(x_{k})/(1+\exp(x_{k})))

    def __init__(self, P0, Q):
    # Set covariances in the constructor since they
    # are constant
        super(StdNonLin, self).__init__(Px0=P0, Q=Q)

    def calc_g(self, particles, t):
    # Calculate value of g(\xi_t,t)
        return numpy.zeros([len(particles),1])
    
    def calc_f(self, particles, u, t):
    # Calculate value of f(xi_t,t)
        return particles
    
    def measure(self, particles, y, t):
        """
        Return the log-pdf value of the measurement
        Args:
         - particles  (array-like): Model specific representation
           of all particles, with first dimension = N (number of particles)
         - y (array-like):  measurement
         - t (float): time-stamp
        Returns:
         (array-like) with first dimension = N, logp(y|x^i)
        """
        N = len(particles)
        lpy = numpy.empty(N)
        g = self.calc_g(particles=particles, t=t)
 

        if (g is None):
            g = numpy.repeat(self.g.reshape((1, -1, 1)), N, 0)
        else:
            g = g.reshape((N, -1, 1))
        yrep = numpy.repeat(numpy.asarray(y).reshape((1, -1, 1)), N, 0)
        diff = yrep - g
        
        lpy = numpy.empty(N)
        for i in xrange(N):
            lpy[i] = bernoulli.logpmf(diff[i],numpy.exp(particles[i])/(1.0 + numpy.exp(particles[i])))
            #lpy[i] =scipy.stats.binom.pmf(diff[i],1,numpy.exp(particles[i])/(1.0 + numpy.exp(particles[i])),loc=0)
        return lpy

In [ ]:
T = 500
P0 = 1.0 * numpy.eye(1)
Q = 1.0 * numpy.eye(1)

# Forward particles
N = 1000
# Backward trajectories
M = 500

In [ ]:
model = StdNonLin(P0, Q)

In [ ]:
%%time
sim = simulator.Simulator(model, u=None, y=y)
sim.simulate(N, M, filter='PF', smoother='full', meas_first=True)